In [1]:
import pandas as pd
import os

In [2]:
%pwd

'd:\\DL\\Live project\\DL-Inuron_Assignment_2\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\DL\\Live project\\DL-Inuron_Assignment_2'

In [52]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PredictionConfig:
    path_of_model: Path
    image_name: str
    class_names: list

In [7]:
from dl_assignment2.constants import *
from dl_assignment2.constants import *
from dl_assignment2.utils.common import read_yaml, create_directories, save_json
import tensorflow as tf

In [53]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prediction_config(self) -> PredictionConfig:
        eval_config = PredictionConfig(
            path_of_model=self.config.prediction.path_of_model,
            image_name = self.config.prediction.image_name,
            class_names = self.params.class_names
        )
        return eval_config

In [46]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

import tensorflow as tf
import time
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

import cv2

In [ ]:
class Prediction:
    def __init__(self, config: PredictionConfig):
        self.config = config
    
    def preprocess_image(self):
        """Loads an image, resizes it to (32, 32, 3), and normalizes it."""
        img = cv2.imread(self.config.image_name)  # Read image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = cv2.resize(img, (32, 32))  # Resize to 32x32
        img = img.astype('float32') / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=0)  # Add batch dimension (1, 32, 32, 3)
        return img

    def predict(self, processed_image):
        model = load_model(self.config.path_of_model)
        predictions = model.predict(processed_image)
        predicted_class = np.argmax(predictions)

        return self.config.class_names[predicted_class]
    



In [48]:
from dl_assignment2.utils.common import decodeImage

In [49]:
image_name = 'inputImage.jpg'
image1 = '/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEhUTExMVFhUXGBgWGBcXGB0ZFxgYFxcXFxcXFxgYHSogGBolHRcXITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGBAQGi0lHyYtLS0tLS0tLy0tLS0tLS0tLS8tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0rLS0tLf/AABEIALcBEwMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAADAAIEBQYBB//EAEIQAAECAwQHBgQDBwMEAwAAAAECEQADIQQSMUEFEyJRYXGBBjKRobHwQsHR4RRSYgcVU3KCkvEjotJDk8LiM2OD/8QAGgEAAwEBAQEAAAAAAAAAAAAAAAECAwQFBv/EAC4RAAICAQIFAwIFBQAAAAAAAAABAhESAyEEEzFBURRh8CJxI4Gx0fEFQqHB4f/aAAwDAQACEQMRAD8A9TKEtskB/wA3yIgMyzFOQbEkECnHCI0203Q4NdxwintGk1qpgDk1I43qpHVHRbLRdslilFA4OGIrg/zh1onpQRQEH3RQrGbtFoYvQ8INZ5yplDhlveI5rNHoI0CVoXiCDQA18HJjqLJWl3dWp5MQHEE0bIJlklqcacYnoTgHDMdlWD0wL8Y1jJtGEopMrZujnwCX4MG5g4iAHR6Qmpu1YuKjxxEW6XLU2hkWcDgrMedIZq8kmhcEKBLZijuPvFZMWKKi0aMZlIUmhzSCmudYYbAUuDLUVfmQaEcdxizNnGaQDmQCAcr2EBMhIJDqScmWTThe9IXMYYIrFXQTtTErAqFAG8MxUVPPhHTZ0qwIKsNpKTTgUtFkskUJSa0JoS+IpSGMk4hw+RBoegZoMwwKpNluTHIACg2AZwxOI91jlps6XYkJSp2ajHPEeUT7RIS4BD7ibsA2S6C6Cd6QR9POHzBctFebMAAKOmr4BQwBo7vDp1lQWokH0er1gy0LCkggEJdiaFtzitY4mbTulsqOw3M79YvInEjIlKlm66VXqsQxL7jAzddrtxQGIVTq0SZpIIISosTkpJHJ8Yj2hYUXVJWDvqaRakQ4jAVJeqSlsSd3OscAukm6Dm6SPQ4QEMCwWWLEXg/RocZikVCA2e6m5sIsmji5QKdnWgbnFDyxaIK0lyyvERJmT9q8U3fEj09IbNUmhZNRiLwIIpUGNIyoylGyKsrB2t1KO8AyapG9vbRLn2nBiz41f5CI85ILluRfz3xpFmckCRJUMwAd9MN8FCSHcAneCMshHNYpPxOnJxnmGMCMzJknk7+WEO7F0FMWNwS3vKAhKgKHyh8x8qPxp9YfYgrfz4N1h3sKjtlUhgHVezwr45Qa1Wdy4fxHyiPqkucScqMemUPE6Yk/ExFHEQ/Ype4ErVLOZ4PHRMDYJfifSHa0qopyejND5ZAfEeHnBYqIyVh3q/AQp4BHxHnDl496vOGKAOKoEx0Qr3CFBVSeMKLyROLPQ55JGcQlyS8W6ZL0JrEux2APXDfHz3U+iySRmUWFRekT7JZLtTllF9bEJQNkNESROQl7xJPukHRiztBbJPpR/FoJrlNh0+30glmAWLwDVYfeJAlcI0i2YyqyIhYIY9DiRXI5Q+9QOX3Fh1qliDBblcA/ukIyQN/0i7ZFAlTHdKvmab6VECm2imIVzTuzBBiVMGD14/YwAp3HN6isTZVAFTHwKXxYvXc16gMCmSUmhJN4uCCMdxGedYPMQDu5HCI6kAULoz/S+8b4FIMQU2UoF9YD/MzwKcF4quqHAkGDzAQAKEHAhxjicYhzZ1xwxAOVCD0yhpiaB3w4ZSgRkoEh+YjtGwSWpV+lXLCOKtKSHwOIxLtk33hkyelR2kBjzHkDWNFJkOKCXKkXlp3C8D4O8cDh9pQcVdjXLNoimYm7dAU2dfkYeFuzOKswI9DDskIJ6s25KTTh8XnDEpY4JD5A06OHhqp68LwPAgZYZR2XKUssEHmklhxMUmJojTki9eKSGzTUdQICpe64obilj5mLZejZyQSD/TifuYrZs4sAQCN4jVN9zJpA5iAzhKWd2akQbRKFDc/twiwQ1NnrAVynxeLjOiJQsgAlzspI4j7wllOb8qhuTvEhaSeQ6GBqsr74tTM3AiqSNwb+avpDEJTyG5yB4xKVIaGLRF5k4ka4HzPI/UQhXF/X0ghlQ25DyFiCTLHLpCKm3eAg4lmGql8IMhqIMTGzJJ4QBSsi8SFy4aRwhWLEEJf80KOGSd0KHYUeoyhWoiYJ10Ux4xI/BQ1VjMeHjI9tyiyqtU4qgVnQSqoeLoaOEFFjELCQ+ZFKkNsyQnDCJGtGYfp6QPUmHGW0bLYwe5xU1PsxzWfpflA5ieURimE5DUSUZqagvAVBJPeDc6wFUylRSK+ZpWzAOZsoAYm+n6wuo6os9TiHcZcIEZSsMfp1iJZtJyJncmpV/KsHyBeJBmjerw+8Kh2BXIUzZA4YQGZJU+bcYlhUdK8oBlWuyPiK8oCbIWFD4RahZPxfP7QwqaGmxNIqxZ68ONIaiW1Wz9vFmTvA6CI9ou1csOcUpEuJ5p2ntM+Ta1rQVSybpKCbySGGKTQ4Yiu4xf8AY3twTMRKmpu3iEli8suQm8m8Xllzg6gQDUUiZaJljtS9SVS5ygkkM7hIIcJmpZsRQHLhFfO7HyQXQpaFAuNp2ILhiQX6x6mlCOq6g69n1/6cck1uz1JcqMvpSwBKjTGvB94558ecC0V2lVZ0BFrdaUis4d4JD1mJzAHxJc7xQmNROlInygpKgpKg4Ukg9QYWrpSi8ZdRRl3MqmUMzCTLG8ekQLTKXJWZaiSH2VbxTZ6OGxOFSYjLtDlgSo7gcBxOAjnlCUTbJHNIW1Ot1QKQWFS3eNQkcWr1h1isy6TFqZLkIDl5jUJAySN+Zw3xDlWNIUpdQVEEsosQzMXyo/sg2f4gqauTDgBQAcBF9EZ9w65I4RFXZeMEVNO+G64QlYnRz8KNwgP4djg8GM/pDROG+K3JaQ0IGYgZlpfLkR6QRc0bzAzPG4eECsTo4uQ2QgC7OMoNfht4b4rcWxGVJPGORJI4wodio9iKYaUx0qhpVHG2jrVnCBAlTGh6lQNSoykWkPCoHMXHITRJVAlqEAUIlGXAlSoVFJkco5R412g0MZU+cl1hlEoCkqZSSdlKFhJSotkSKA1j2C12qVLIEyYhBOAUoJJ5AmsZLtlbrOuXSaUzE1TMBCUDhMvFloPItiGMdnBzUJU1szLVpo8uROJ7xKXe64YKDkAn8pwocHZ3pE6VpG0IomdOS2QmKbwdoZbrMu0S9dcBQFXFrQsKReOyFpL4FimtaCmcDsAXJKSlZvIIUL4BIIIIGFRTN46tTTcXTOZM0uif2gzLOCbXfmy6MQE30l24BQ5l41uhu2NitRCZU4Xz8CwUL37LjaPJ48jnSpqyQoIKDkCX54U6QXQejUyZ8qa6iELSpqPQg0wrGS4fSndujSOrJHsmmNJIs8srVXckYk/IYOTv3sDirZpSdaSK3Q9EoFCeuJ4+DRZTdIy51tsaErJBTPJYsQ6AEhSTiDdXsmlIjdoNFrlzAJCkS0YKYG+HNbgLgAinC8aUjv4Dh9OGmpuGUn0/T4ydXWbdLoM1K0JCjPKMmvKPGoALDi7Uik0n2gmJASguSSkqIAIxCWJ77lqJrziv0lapkqaUGSZi6EFalqSp8CkICSNzBQGLpxgdo0IubLM2ZK1EwKKnuKEpaSCoJp3F3gQHYG+A4YR1a3Ca8oKTUYr59zBaqsFoO1qlzxM7iUqukqBBDuFC4C5oFJZqEioMeiaO0kicUXSFuq6WwyJbF6R5va7IZguoMoqQpJUUso3S4KypICSAQGCX7ya0rb2TSyJVxC07KaNeZ6Fqhi71jz46Fa8YPraNM/pdEvtiZxvSgsKuL2JgUU3UrFZMwu2Q7xozuxYehdiLNbLPZ0otASkBmur1gLhypVKOdxOBrWM/YdKWScgyiEpSfhYAcw2cWHZ3ThsaxZZyr8hX/wAE3Fh/DV8vDc3tcVwmbyUfqXX39/FnJDUrZsD+0GbMCkG4m53FkY3ZlCoOaNg1cR0z/ZyXsrW67ylMUE7CWdro4g7sSeZ9D08hEySVIIN2o+kYbs1ImzlzZiAGKheSTklKrxT+pynm5jl1OGhPhskqcf8AZtGTyomKeB3IsPwpzhybHHj7GlMrWPGExiz/AAkcMgCGmFFfdhJQd0T7ghBMAiCZMc1cTlIENuwDISkNlDbsTSmGFEMREMuFEq5ChgeoXo4VQImOPHnbncFKo44gTxx4VDCPCJgbw0mFQBXjHaVsmlzaSZFokJkvshaXATuKQm8Vf1V3jAau9FF2r04uyytahF9iAUspSjefuhJqaYc6hoqNiaPIO3ulbWi1mXOnIXMQWOrSyQkgFIIOALgsd+ZMU1g19ptciShlGYpNPgYKIUpaUEbICVEjcIfpvSKrTNmWqbQKmFIU2AAF1DXsgSf6jyGi7P6YnaNs9onIsZbYeZPC5RGAShKVJN+8pS1MFUarbMdu6jsc3VmhsnYa0JK7y5ITW6iWgAKdnStkgKSQMFXssw4zWk7DMkrMu4xH/TVeUlt8tYBKBhSowAYVj2iRNvJSpmvAFtzh2jO9tE2cStZOWmWoOELNHONw0qCw6tBpcRJLGStfp9ipQXY8qRa5ZcPcWMUro3XAjiDBky1UfOiQkhSlnJKAkuSY7aZqVELU0ySpmU3cPEioSaVq25qiTZNAoBC5R1ZBfZZ3bekjI+cXtZmT+x3Z+cLUmctKkBLlRWCKqCgwcVZ8Y1ungkqBBBpVi/pGblSJmCppUMwRQ8wXhiLBJSTqxLExvhCUq/2soDl9x6unx0Mls9lSXxmTgSdI6YVfCZTpJ2VLSzukHu9VVIp1eKnTMidNlqRrlklqKL1cHCgw3HPGNDZdAS0pC5kxYVgyQM8kpYmpOHGJA0HOIKlIuB9m8xU2V67R+Ajs15JxpPcyU0tjz6waM1A7iifiUEuTV8UqLJ4cIdPtklik7H8ySK8bwY9Y286wKTignixA9IjzZQul5b8HBPIChJ5RwR0HHdIbmn1Zi5Oitcgqsihrkl1SL4urSfikKJoR/DO+js0RjpSagmTaELSoYomJKFp3G6qvUQfTdkSFFapSpTVF4oCuGzevJ50iHN7SqVK1C162UMEzNq7xlrcmWriPOoOmlxetwzq9vDDGGoiRYNPWtMxaBPBSEqWm+wSUiveAcUf+0xoP2e9rJ2vUlSUBCLLMmJRLTVc0BBS5USSoqN1nArhGEXMlmgmEUNVJJZ6FNKKBGbCuQi67KXZc1ChaEC8Uy2I2wlU1N66gg3i15t5Ijk4jWlbjGbcXv1+dDaEV4PW9HSmlIS5UEpCXOJui654lolXYOiQEgACgDAcBQQrkcVFWBuw0y4kXIVyCgsimUIGZcTrkN1cVRNkLUw0yInGXC1UOhFeZMN1EWOqjmqhgV+ohRYaqFABqSY48B1kLWxxYnZYZ44YCZscM4QYhYaGmB62Oa2DELCRFtNjSsuWphSocMWOWJ8YLrYaZwikq6Ce5hldjLDYlqta1rmFF+amXMKbiVOVX2CX2WoS7MKEgRhrd+0HSFpUJcppd4kpQiUlRITtOTMvOwDkgDCLr9oVnlLm2hRSCVJB1hJJSZIOwkEsKpBYVLqGbpwmgNJLkTUzJTBSVJLkPge4HOBzDh8MHjohByaXVmLdG6HbjSkuUAuXKUa/6ikEE8wkhPgKvFfae2M+dMlm1IRMlSnKkAqlyys0ckOVFIdhvehxhumrXIRa5+pYoBvBSSFSwlQSakC9skkMWrTAAmv8AwyV3QNt2wwAXUEJHdfz3nEvl7XQZMl6BkqnzrklUiXJmrIly5pN9Q2iQgJG8GmRU7YiL1VplypSlAABCVFgGAuu4YYM0Us3tUUaO/BSpeYIn6xlJdSV7KUpZ6NRWb1LvEsml5k6UqUszJs1YCEEqEwklk3VKZ0hrxdROADgO09xEqy/ibSWJKQXZKcSAHL5AAYnLeaPZW63S5BTIki9MZglHeUfzLViB+o1iZKmmVKmSkEa6ZRc4HugEES5Y3DHial+7EbQ2jZUgEgkrVVS1F1E8THocJp4POX5GU5LoX3Z6xrSQuatSl4gBarsujbNcas/GLPSlsat9TszuCWd2BKXGWGMVFgXrJlzW6sMS+9m2U1DkvvyOOEZztUJkualCFOCkKUo1KeBD47sOIjoetp500Z8va6LG1dopiXCVcm2WxLshhiSaisUXaXtpaAhIFLzgFLgFmd1O+eAZ4r7VMNwlKkukhxeF4ihIAd3Ip1iTp2ySlCWhc2U6rqkbbkhboChdBASyr1SHuxhxHFW6hsKPDQbuSMnpRQWygpZNHvM1Q9GwLvllEAJi0Oj1BKVAuksSAKpqXo+0AXw3ZRptJTbHZJSEahM6ZMDm+Q7YXrzG6HoABVjuhR4L8N6mrKuna7vv8s2z3pGKRL4+Rj2b9nvYdNnSm0zrq56g6ACFIlAj4SKKW2KhQYDMnzRgs3pVlYbkzbw8FJdvKPQP2e6ZmBRlzZa5KSdkXUJlKJYMyUi6XOIDGjl2By1NPRUU4Tt+KaHb7o9EuRzVx3WwtbwjHEmzmrhauO63hDddBiFiKIWrha6FrhBiFoVyOauO64RzXQ8RWhauFqoWvha+DEMkc1UdjmvhQYsMkG1/v/EN/ERmF6SteOpkf91/RBhfvW05y5P9yj6S48znrw/8fueryH5RqRO6xxU2uXi0ZgaStH8OT4r/AOEPGk5wHcR0H/JoPUR8MPTvyjQLmNmPGGC0jf78YqBpOa/dQ/8ALDv3os4olnmmHzok8llkbSI7ItAKkgihIdsWesVZ0kr8ksf0iOC3qGAQM6JEVz4+BciRku3NhkrtU0SUgISopUQMZgLzCTmbxIJOaTGG0hoxSMMPnG50xI1UwkKrNXMmM3xLVfU/9Sy3t4IUFUU1eHyzjshN7Sicko06ZVaGVYyhSbSlS0KukBBKVpmJ2WWlBDggggmlGxMXfZ/RssaQUhMkzLOJamv3mQUygtBUoVSHBAXxfnV23s8lQK0FlJSpaeaWNDkYtdA6cnLlJs4KDOlpu2czbxQCARq1oSoJJUAyZigSGunGLlqOTbfcVE3tB2Ys8uXNtcxKJqFakSLxWlRCgXlztWUqEy9MS4GNwOzsM32VSkqXMSlGyLiSlIBvK7w30SKvU3k4NHpWg5arZZLRK0hZk2VAIEwFSRLUslS1T5asUKvKBxIDJY4gYfR2j02SaqzyZ0u0hSiqXMQQRdOyxI2b2wXLsxyjMBptABVX4lJJGBUmig+DjdDTMO+LCwTkSZeqWgFJKyo0IKio3nHXHhGe0jpOTfuyEzCdzC5ngSbww3NHoR1FgnIwa3pE8qMRLdOupUrcknwEFlzXFQQoYpOIPvOIeku4v+U+hhvdbMO+5m0EAlRJJeoJoTjViDXpCkSypSLgQFXwb2d7EO5auVMoeuXTOvAFomaMsbKDsHrVsX2WG/HPHOOGHk3NVpPRaVWeyzpKVHWyytQxOsBVrxTAJmBfJ4ylosZWJs0rDoQ6UqqV3e8Ek02Um82MarRulbtlVZlpWFi0kylJIISmeliFG84Tfrm7mrRUCRJvzkrJMlBId7msSghQUA2yoC6psWUocD6nP5nA4v8Ata+ePt9mYYVqWu5AkzhJmCWssSmWpE5IZtYhK2WK3ki9dfJjvjQ6M0vq1FC6JLpUhkkOcw4BOLghQPGIvajRAkGzTZslS5ZTqVC+1UA3RssQq6SRtMq5kAYi6Vs92WhaQU3Wl9EoSpJBzDEpc12I4dSp3Jfyao9N0bpR0MV3iKAjNIzNHzbAYF4mpt/EeUee9j9JKK1oDElAIKg9U1zDZEdRujQjSCzS7Ky+AfLnGK1EtmhPSb3TL/8AeAyPlBTajuHve0ZtWk5jtclHmiBr0jMApLk0/QT84fOXhi5D8o0yraWwHQ/eOC3c+dIyh0tP/gyuiAP/ACjv79tIGzLlh+QHkYnnx8Mr08vKNYbanieL+rwxNufAPzaMkdL2rKTI8P8A1hqtK2z+HZ/AfNEPnrww9O/KNebcNxfczR1VsAoR5/eMcnSVtP8A07P1UkeoEOOkLbhq7Gf/ANJXzUIOdHwxenflGrNvTCjL/vG3/wACx/8Ael/8oUPnLww9PLyi6kCp6fP6wcCI8pWPOCpVHmqJ6rkPGMOmGkAEyvvhDZk6DELDE1hpXj73RHXPxiFM0mlyBU7h9cITQ0WBmYcj84ZNtSQ9frECVKmzDuHDHxi60foDM1PvGkJRb6CckjNaamXpklRBACiHPEy/oYyXZa9rTJUXRe1bEtdWpd1JBYtWjD8wpmPS+1+jR+HZLXkKCg2Jd07v1Dwjy7S0sotImJcCYdYGLMVUmAHeFOXyoY93+nJS0ZacjzeIf4mSNRpuxT7MQheYLEVSpN1JJSSA4DirBs90Vqe0E2QiXKlrUEh1KJAZUxR2gzlkABIAo+0c6b/sf2kmz5a0XWmyhnS+GxAI2agpLU5OAMpZ9HCZOH4pJCbWFKkzUbKBMSTelMxALBw+LZl455aMo5J9gW5Zdl+1A1c02i0WRAU6dRP1iLwLXVJWLwIDMwD5kij0emBYUKlmx3L6ib6JU9U6WkgO6StIUmpLYAjAUiBap0kHVzAFJ+EtW6e6aUqGOWMQBZES1pXLKlpUrZCA5Tsu5Jpi/gOUZgStIoXqnSkrBUq8lypVVFRF0FykknCmWcQrKZdyYJi035gCAFq1ZBNbqnb/AEwbqnNHY/DS5M3u0UkbmxF0gBwFAVY8g1HcNWpRwIPB/lHQ4KdUzO6I2g7POnI1xF0IQAVrN0LYn4WcmoY0rvehLVUF4WvUNk0zbLnDJhJjaMcFRL+rcrbEhhFjoyWFTDeDi6oYhJDIWpJBvByFMWd2eIaaKbf6+xEqyJN8AU2kh/5gtIPiRHKoPmKPuaXtZyfMllU1BW5mAgkkgBQF4C7gC90OABjhhEW1acmpQiUiWmXNCpk6ZNb/AFFqW+waYBgACDiMA76f9nlgROnqta1lJlMoIugqSClQTtEMqiSO6AWIONY/aTRty0zEzlAzJhSshNCDtAFDvfJJIo+JxUxgxnGTg/z+fmPbqaq2S0WnRqlqUFmaj8SlKQaGUmUV3AQ4eY7AvWaUvWKHTejVKsqpjXUpZJGJKkKAKlAUQWxS5NathGh0DYpcqyqlNctCJJQsKVeRL114oBODruJWz7IKccyab0LLsujp8tClKvF3WXdS1pwGQ5Q1LovcaPL+yc+7aJRNASEk8CpJPgHPSNqJwcgHBTNuYtGC0Ulpktt5HGrAN1MelW/QkuYbwULx2qHM1LNHPIqJHQuGqwMRZ1jnSj+cccfGB/jxgoFPP6wrQ6LRUNSmg5fKBInA4GHJmYQxBSiH0aA34ek0HIQwOpQHhTZAcUrX0h8vlHJvw+8jAKxgs0dggVHIqgseibjz+Qh5nRUL0ihDuavgKnAboizNILX3QQPOOTY7LLg2oA1OX0iNMt5PdBPExFs1lUrF4vdH2AbsM4WNg50V0qxTJhdTt4CLywaFSGpE6zSwMifD5xZyE8x4fWKUCHqHLJZAMosEBoAFDK976w9KX/N4/eLozyO26XfQpN0G8CCDuIYx45p7RRClSZlCCVIWzsTQls0lwCI9hIbI+P3ii7Q6FROT+VQqCTQH6Rto6r0p5IiayR5v2cFts0wLEtS0uWUnaocWId0lsDWgwIEW1v0u1j1C0slCySSQq6EqJSU7pit2KbxJ41ekbeZKihSJKiB3qEHH3liIztv0iqYdohhRKRRIB3AR16vExnuo7/P3MkmgNunlRJzx+3yjf/s10ENUufMQCF7KAoAggVUpjxp0MZbQPZ1U1QVNBRKHRShuAxHPwj0mz25KQEJSyQAABQADADhHFJmiQW0dmrMokmXd/lJSOiQWHhEKd2Ss5wMwO+YI8xFh+896TBBpFNKH2ecTYyildjZJlJCyvWsLy0qeuYDhiH4PDB2OQP8Aqq6gE+TRfp0kjMH31jq7Yjf79+kWtWXSxYoydq7HPhND5bP3iHb+zhkpviYVksO7dunEHE/Ew6xtVTknB4j2mWlaSkgkGh+uPIw83diaRiNA2wWa1iaR/pTQUqG4L7tMrq3TxBO+PRhIlT5mt/0ZiUpliStUpExcpaSHurUTRTCiqpVxIfDzbEAVIW2LgkXkgnFxmhWfsgcjSGqvypqQhMwquqDMkrJNFFhMTuI2mAdIaPT1IriEtTT67X8+WZL6dma/tTpAqlgJCVhwqbsp2k0JwA2nCScQAgCpeMr2gMuXJZCUJvbWyAlwK90Y7V3LOLBOl5ak3g5wehAB3cS+53jH6VtmsXdTRIwG4O48zlwGUcsoPTi8tn29/wCCrsZoCxayfKQ2Kk4cVV6MAXj2X8KkYJA5Rh/2c6LdSrQoFhRApRxXoBT/ABXekjjHKUQ59mBint+hwrKNEtsQ/p84CUj9Xj94TGYW06FUiqCRy+0RF2iZLopLjfhG7tMtP6vfWKe1WUHJ+bfWFQ7KOz2+WrNjuVT7HxibJVQch6RCt+jKuExXqlzZfdJ5Yjwh2BokmHKOHP5GKCTplQotJ5p+n3iws9vRMIuq3+m4w0xUWaYUBEwQosRT2XRxzEXlj0Zw8ok2ezDMt1ixs6Ej4x/dw8o58TVzbG2awNFhKs49+6wktgFV5v8AODJWkYnz+bwyR6ZQpj75QdCDAhNTw8T9Y7rx+Yf3fJ4LAkJT4wZHv7RBvjf/ALvvBZax+Yf3e/YgAlXR9o4qUDAiU7xDnScLvl79mAZFtmhZMzvy0K5pSfURVr7HWTEWeTz1aX8ovLwyZukOE3eQOo+fusFgZ5egEp7qUpG4BojTtDkb41lGor0+nKIU1aQpiRm7sNwPz8IVgZY6PWxxpAVWWYGf0jXjV70jkab4dqZZFSP7hvHHnBYjGas8YYUERrZ+j0YjFqV4hvfCA/gAffgIYGYlTC+ftj6Hygi5hyi9Xo1GJbx6hvEw1WjUivkK9GHpwh2IzVscipYjA5cjwjNT9KpQopLpP/1TGSehj0r8ACC+WOArh9D1iPN0RKV3paCP1JSfF4uGo47p0Jo8stOnc0Ehx3lKvrYs93dQtSJWg9CLmkFQKZYrWhXyGLcY9GT2bsgqZEl9+rS/kINLsstHcSkcg3kIcpuTtsVUN0LZylsUpFAKeDNFuvn4N9IhSpqTz6sYdrRWrbwT9TEgGUP1Hygda19PpDdYPzef3gYUPzefpWkADysZv4jziPMHD0+Zh6pgGfvnDCtOcAEadIB9iIM2wA8DyizKknA+ZHzeBTK5noT9YLAztq0T1ipn6OIwBjXzh+p+sQZ0oHPzPzgoDMayaKXleJhReKsSePvrHIANdLPAjw+Rg0ud+k+X1gRpnDkrBYeXpCGSUT2+BX+3/lBNecbqvLwDmGBN0byYYqYDRwBmcOQESUFTaCR3SRwIy/q91g0qcSe6rybLjX7RFQu9h3d4+TUZoeVDAYZk5/aAAyLQ5e6VHcCGGWL/AHgxnHcSeY+uW7CucRLzUGJz3ff68ocJrUHjv5e/lCsCT+KLd1XiOufTrDNaTv8AL6+xveBlfv375UhyV+/fvKEB0TMmPvrC/Eh2ZXgff+YYVNn7z8PKHSlOffX1D8xCAKFnj4H22XSEJgAzfkeQy5wxKyD7pw9fCEJjljT7EBQ6PDARnMc8W6ezCE00NWcHxUD6CI9qWWwxbzYH1gyJt0Ma4DkNr7DrAFHTMp74wJTbvZYnyMSJo+b/ANx+URjPwOQ2j0o0MR1audOBz+5hyZ6c38OG/q0BvGnh1Ll+jDxgSV5ZHDlgPMHwhASjMq4J8CMKeB84YSMX9feJ8zDUEkZvy8DzyHKB36v73U6lhxL5RSFZybO4K/tVz3boFruf9qvpjBZkxjw94e/ywNSXwx/x8vluYsQMqJOyDyrVuYx3ZF23QUA7vTwgS5u+nvL345IrJ6Z+/ecMB6njgPE/LjnDDNPv378YWt4Hw8OsAHVk/lJ8PrhwhqCfykVyb3/iEZ3A+HnHBaMq+/WABsxY/Ko+HzMMQRmhQ5hPyMGK8gXP+coHMQwzPhXrh76QIBiwnJ/BjA1NmD0/z6RxauBHz9+NYZfB4/Lww8s4oQwo3Dy+8KHFXEwoNgslIs8vHVy8x3R4Ybh7MJQS7hCegA9PfpHYUQUdUhDsUIJzdI51puB8IcLJKd9VLx/InGvDga+HHkKExhpdmSnuoQFHE3RQYEilch4ZCHCQjEIHgMzjhi7xyFDEIykkOQDhkN9B4+dYYqyoJ7qXP6Rwryw8oUKJZR2bZpYAaWhzndD48sXfyjiLLLbuIo3wjdTyc+UKFCYhwkICXCE0FKDJiMqf+3CC/h0P3E8dkb8fEkNChQDClAyAflxb1HnHJUoEOUjLLkD6woUADVITQFKSHbAYMUnzDwxCUKPdTm+yMbxhQoAHKSlOCU0D4DdnALXJAYAJA4ADCn/lHYUA2PlpDMAN9QDQlh5JJ5w3UpU7JSKbsmBHVn6qhQoCbOJlJLuhJ4MKua15lupMDm2VB+BJfHZAerdK05A74UKAQNEpA+BDNiEjc/hm3EDfD1S05oS5r3RiKkYe3aFChoCPNs8twRLQ+WyMONM384b+GlmurTl8I48PeEKFDYhkywyTjLl73uBizEOCK5f5eOaiXhq5YqKBIofCFChoBiZCHJupcHNIYmCS5KPyI47IqMRlChQAPVZ0VIQj+0cK+XlBRKTkkcx94UKBADVLGQA6QCZLScQD03e/KFChsQ38EnKWk/0p+ohQoUOgP//Z'
image = '/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAJQAzgMBIgACEQEDEQH/xAAbAAACAgMBAAAAAAAAAAAAAAADBAIFAAYHAf/EAEQQAAIBAwICBgYGBwcEAwAAAAECAwAEEQUhEjEGE0FRcZEiMlJhgaEUFTNCwdEHFlNUkpOxI3KCg6Lh8URiY3M1RVX/xAAYAQEBAQEBAAAAAAAAAAAAAAAAAQIDBP/EAB8RAQEBAAICAwEBAAAAAAAAAAABERIhAlEiMTITYf/aAAwDAQACEQMRAD8AorRcYq3t5CABSMMJ7qsLe3O2a7wWELkjFPQJSkMPDjFWFutKpuBCeynYkxQYFwKbjBrJBY1xTMdDjXNMxris1dFRaIPfWIK9YYFYNRc4G1IXA4iTTrnYilJTua1IaSda9QYorAVECts6i3KlZUzTpFCdKCsljpdo96tHjB7KC0PcK1oTWLY1HqMnlT6QnNHSAE8qlqwrb2mY84qRtQKtVjVYwAN8VAxcRrDStEfCcVYWIjBBevGtSzeiBUXVojjFZqr2F0ZMAbDlRMZ5VT2sshIWrJXwKyzY5hFa43wKOkQB5VKJs00iZxmvQw8jhPOnbeInsqUS7U1CB3VKokMJzTkaYrIyMUVayCRpvTCpQ46MDWRIDFeSnG1e5oMr70A3NAkNTkel3bNaiPCaxTUM716aomWFRJBqBNeA70wekCvRFxchXqqWpiP0NjQCWHHMVNUweVHJBFYozWVQ4c16FxRQtTCZoaCqnPKse2496cWMd1elQKzQtHGIhjmawtipvtS8j4O1MXWgW1WsHIVVWxFWkB2Fd3LTa9lMxUnIszxYt5UjfPrOnEPLNQLajHLbxCW2ZpGYM/VMOEAZG3Fv51nF1dRmjoarLCO+Rm+m3cUy49FY4ODB8cmrBWFZXTKtUw9LcYrA/vphpsyUP05TiJGf+6M0neXsNpazXNw/DDChdyO73e8nYd5IrSr/AKQ3EzGSSaRQ26wqxVVHdtz95/DACmt7kgdPtXiiH/kkUUrJPp8RxLqtgp981cxk1W9adGEkbIjBuq4MhvHvpi81rVrqJY1FnbKCDmCAI3nmiXykdAOo6ON21qw27mJ/CvPrTRc//NWf+r8q53c63rc13Fcte8MkIIjEdvGFGeeV4cH4ionpD0h6/rTfv1mD6X0aEDlv9yhzjo/1pon/AOzZfEkfhRE1HRn9XV9PPhLXM4+kPSFZDIl8eIjGWtYuX8NenWNYdEWY2E6RnISWzjIz4gA/Oif0jqkU9m/2F/ZynuWdfxo5ilO4jJHeu4+VcgvNZuLqKSO+0zTyWJKzxQGN0PuIOMe7GKUsr29tCGtb6SPG+FkK0xecdqQFhsKKsbVp/QjpW97KNM1N+K5OWglY/ajtB94+Y8K3bOedStIqlFVcGo5Fe8VQELYFBlkA2yM92axn2NI3dlZ3RJuLeOQ8QYkjcke+mGiyOFGWIHiaXc5O2/hVTaaRbzC7TUII7iFblup61i7cPcc9xzgVZJHFBEkUKiNEGAqjAArWGuf2kucVawTAdta1BcgDY07DdHvrtji2aOcd9SedBPASw2LdvuqliuOW9e3QS46kSKHCSBhkZwRuKmLrZ1nFTEw76p1nNFWUnG9TF1PWOkFlo8cb3rP/AGhIVUXJOB8qV0LpVa65c3ENrFNH1KhiZMbg57vCltY6PWGvyQ/To5XeMYQxysh+VM9HP0d2+l6j9MsNadLXPDcwmQScWM+juMjf31i3K1OyXS24uL66h0ezVpDGBc3WCBjb0FJP8WOe61qmoWtzCytcROnEcKW5H411LUoOigmuJPo8aXLgj6asYcq2MZ9LIJG3OtSuNKNweG56SJPDyCPp0ecd3Pas6WNQUOvFwZDAZO3KsSSbIZSxH9a6pp190W0Wwe306zt4Sy4f0VYN471KLpD0bEahtOtQQNwOHFNqcY52mWAJHyociyA7ZI7gM/0rpZ6TdHV9WxtPJaieluhL6tjafwrTaz/P/XMjkAZyPHal7t3KqkeT/drqJ6WaA3r6fafwCoHpH0Xf19NtT/hFNp/Oe3J167BGWI7RUTtuQc11SXpF0djeM2Wn28LbhpuqDcK9oAqn6Qp0dvSsmimG0uM5dp4BIhHdwE4FXkcGnW0d86CWxjmaa3YSxvGhOCPCux6HrEOraJBqSnhDx5kUfcYesPga0KwMDTImva3Z3FgowbeOyUE9wBztWzx/U8tjc2OgSvpkk4/spHHEOLGM8B7CNtqWtyWFbb9JWiTORILiIA44mTatvS4SSNJEZWR1DKQeYPI1zGL9E+mwDN5qd9JIoDMkN0uMHtxwbefbW7WvDa20NtACIoo1RATkgAYFJ2bi3MoPbQZJR30mJXNRZjneqaDb6hb/AEme0EyG46xnMQb0gO/HdR2k3qus1UXl42PSL4J+FMsaqObwxe+noYB2mkm+s4F4mtbUrkZPWt347qtjHc2zIJI4JeLkI2Kn505xMHhReyjlcPEPaai6eguULiJ0wcENU7heDU7KArs4d/LhH405nE1HAp55owiCjYfKm0t/dRVgG1OS4Q0/Vbax1ZEuwOAAlSWAyw7N/OtT1npE9ldXnUSl4JZi/GrK3rb8gTtkmrDXtNbV7potMu7MX1jOXNpcPwdejLghT2H860OTTZdP1GOC5sZHtFmBuEjZGk4R9xSezPbzrne63F/ZdIrqC3M80Mk+mTEh+KNuBgCQcHHMEHypG/bcXFiM2cuShdssvuOK3b61tLvQkuGgNrpEKBfojFVaVh6sZAPooMAnv8B6XGulnSOTWLyVYESC2D5Eca8IJ79qkpe162oxD7W8gXHYD/vQW1WxG7X6Hwx+VaRgbVYvYxLokd8rkytKUZPZGavJMbJ9c6YPWuyfDNeHW9L7bp/gDWl591EAXHM8XhtTkZG5DV9GP/VyfEGiJqejt/8AYlfEVo7YHbXmcdlXkcY6CtxozJkaxGCfu9v9K8kuIDgW1ws3vHLHwP4Vz/Y9leZ4TlDg99J5pwdDtruOGUPMvCM+t623u7jRbq6UXIaeR7dn3VZEYHHLbate0XUpLqWJus6u8tsPFKowy47Qe/8ArW9dKNfs+kPR2CHUbS7TVEHFDJboCgceJ9U+z2fAcWr5eokns90J1Ca01mR3Ms0ci8MhYY9Hbv351u8rR4V4DmNhlSDmuK21hqfUmcQ3KTyRsg62YCOMvsWHMk9wrrGgRMuhWCNIJXji4GdTkHB5/wBKkvanDLUTLtmvWQ9xoLp2kdnKtamFreXhuro97CiNPVJpE7zaldgiQBt8uPfsB8KtHi/7qvSRrcsouISqZJyNviKcu+KdlaHOFznO1VMEbH2PKrCCDI9VT8K82uuNg0N4La14LluGTizgDNL6l1t1qKTQwxFIdopOMhsHBII8RS8Nux5RfOnUtmx6i578VZSxsaXmnkes38BogvLDkC38BqgSBu5fKpiFlYElMD3Uyji/Si/Nzrd9MmWD3MhU8jjiNLLqd2vD19zde/jYtwj40tNKYxeXAOW698E9mWbFXGl3eiz2TyXl3qDXKISwVVCA5wDkjlyzWogMeqRzL1dxO0kRYZUpzFVc/Ry461jb3No8RPonrsbdmxq66SWWg20Uz6ZrK3M8ZTq42jwZAccW4GARnPvHbVAl3GFGWAz31d9pif6uXw+9bnwnX86kdD1XqhF1kZhG4T6SvD5ZxQvpUXY614biP2186fFe0v1d1Hsij/mr+dZ+ruqfu6n/ADV/Oh9dH7a+dZ1y+0POp8TsT9XNTH/TqP8AMX869/VzU/2MfxlX86EZl7XHnWden7QedPidijo5qXsRfzl/OvH6PXy7t1AH/uX86GLhO10+VSFxH7a+dXr0djWNqumO9xdmKQheGNI3yST2+6m5dZ6xcQh0HezZqtVknniiWWNDI4XjkPCi5OMsewCtytdG6KW0aCbpBJdNNOYg9pboxOBscMSQD8TV30mNWm1a7mUKno5O2D5V0j9Dt+Hj1S0upW4oGjIzk8wfyrQ9cvbRbtLLTL2e6tgwUmeNVwc42x/Wtm/Rbhda1VM7tCjHfG4Yj8azbVdbaW1/aHyNDeS05mQ5HepquLMvLB/x0N5G+8AR/erO1VBp1teWeptMycSZkw2R6eTkA+H4Vex3Vv1MZn+1KjjAHI0GRlP3gPiDScoTi+1FTx+P0t+9atFeAfdPnVjb35AGM+dUEUij7vypyG4A5IfKtI2WHUGOOZ+NOxXynY5B+Fa3FdjYcO/vFNxXDe3jwSiNhW6Qj1T5160ylGwvYapo7xu8/wCJaKLts4L7Hs4aK4/FaXmow3NtYwNM4uCxVeYGT+dX3ReDX+it9FqUnR+SXqWEgLxb5AIxn2fS8wD2VrLXYtbm7ie0t58zN9rx+ic9nCwrE1i5Q4haWL3R3EgHzJoLLpLf/WN07Pp8MMrScZkgiIZvRxjJ3O+fjSr2PW2/UqySNs0L44ST2oQdx7uwkc96WfVr1x6dxcMvvmJpZ7gOcmPLe0zFj86qMNlcqxzbTD/KNCaCZfWikHihFHW/uUGFlkGO6Rvzog1a9A+3l/mN+dOjshg9oPlXm3fVl9c337xN/Nas+ur794m8TIadHat2HbXu55A1Y/XV/wDvMv8AMNZ9dX/7zN/NNMh2SjhlblFI3gpNMQadPI39pG8UY3eV0IVR30Q6xfNzuZf5rfnQJL2aXImd5B2BnJ/rTo7M30W/EF6tc4WJFyQvLLEbZratD6SLZaZd28GhwlbqAQBoQSynAzIB3nGc9/hWmpdOn2bSJnnh6MdXvyoU3VxgdgkwPlQTvLO/kvpLgWEkAkkLiNFIVMnOBnsHKt0/RpIh6Sap1uQv0cY9x4h/vWhm/kJzKXl/vysa2XoBdPDe38yHhLKAfixNB1x/ouMl2FLyG0A+1atfOrSkek/yoL6m55uMd2KKvXa3HqyN8DSUskfF6MhPxqpa+PPNAfUGz9oamGkUkx2DzphJ8dkY8TVZxEbDioqMTzz/ABCiLeObP3ox4EU7EUIGW/1CqWA75PzIqygk2GAMeNBaxSjHrRgeNY8wAPCQfAilFlB5k/A1PrBy/qaYNG1ro1PPf3FxbYRJXL8B3wTz+dVEnR6/T7mT7hXVECueXzoywow3ANUcfaxvIo+ApsCTypdracc18q7X9Chf1lHyqLaXbMNolPvqYOJmGUfcNRMT9xrtL6PZ8jCuaE2jWX7BfKrg42IpPZNZ1UnaprskOhWcsyAQrzzjA3xTt50csY7V3+jxhsZ2UVnyuWRZ9OHdVJ3Gs6qT2TXWxodpy6pTv3CvfqO1G3Ur8AK1ia5H1MnsmvRDL7Brrf1Ja/sQPKoNo1uD9kAKuJtcrWCbsSvfoVzIdkrqX1ZbryjXyqL2EQ5IB4AUw1zNNIvG5IPia2To9atYQuHX+0kOSfwrZGtV5YqJtVG5FMUqZTjlQnlOPVFOPCmPVpaSJe6mBZrgDt+FAe6Gea0d4R7JpZ4BmoJDc+lTEMYO+QKHHGTgsAKZjj3qBiKNBzINNxBaTCFeRqamTNBZIyLtt50ZHj7KrFZzyGfhRkdv+RVFsjx5HfTChWGwAPZvzqrjfh+7g+NNQygDck57e+qHgoRQTw+dT9AjvpNJHIwCeHvokYfByx8qAvBHn0S3voiwg9poI35nJNe5CDJyd8YFKLGxgzMPCnL2KPqJFJGeHO3uqu0su0jtxbjAXPZVjOrtCxaTIAORiuHn+nXx+lOIYs5H/FTECt2r50NAFXi4jzqfo55nzru5Jm2UdqjxoL2MfDlnBPjWSKCpByfjyoTSNt623uzQBktADtk+FCa2C80b40zx5OMtnHhWMw3JY47s5oEHjX2TS8ipnYGrBmXHKkJ1LHAHo+6gWcdyml3TP3fnU5eJdsHFAYHsLedQQaIHsPnQZIRn1TRCXGfSGKHxsOfCfGggNkFZxsMYNZWVARHYjnR42NZWVQeJzkUQuS24HdWVlAJZG4sDup2KRmJBPLFeVlA3byOXTLHuptWbixxGsrKomhwSe2piMcPFls8udZWUFhpG6sTuS29G1h2jtpODb0aysry+X7dp+VIrliVbcbbVF3bjx2EVlZXqcS7SEOuAPSNBN1KVOWzuKysoCK59E9+T8q9DEsRy27KysoIyqOA7nnypF9nxk1lZSgEsrIpK9lAZy5HEAcisrKgHIAFOOzlQWr2soP/Z'

In [50]:
decodeImage(image1, image_name)

In [21]:
import matplotlib.pyplot as plt

In [54]:
try:
    config = ConfigurationManager()
    eval_config = config.get_prediction_config()
    evaluation = Prediction(eval_config)
    proces_img = evaluation.preprocess_image()
    result = evaluation.predict(proces_img)
    print(result)

except Exception as e:
   raise e

[2025-03-04 18:03:56,651: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-04 18:03:56,668: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-04 18:03:56,670: INFO: common: created directory at: artifacts]
1/1 [==============================] - 0s 70ms/step
bottle
